In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

import shutil
import os

from sklearn.model_selection import train_test_split



import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import time




In [ ]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns',8000)
pd.set_option('display.max_rows',7000)

In [ ]:
!pip install tensorflow_addons

import tensorflow_addons as tfa

In [ ]:
train = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")

train.head()

In [ ]:
TRAIN_PATH = "../input/plant-pathology-2021-fgvc8/train_images"

train["path"] = TRAIN_PATH + "/" +  train["image"]

train.head()

In [ ]:
def get_list(string):
    list = []
    list.append(string)
    return list

train["labels_long"] = train["labels"].apply(get_list)

In [ ]:
train["labels"] = train["labels"].str.split(" ")
train.head()

In [ ]:
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(train['image'], train['labels'], test_size=1800, random_state = 12,
                                                      stratify =  train['labels'] )

In [ ]:
data = train.iloc[y_test.index]
data.head(5)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import time
import os
import numpy as np

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=45,width_shift_range=0.15,height_shift_range=0.15,
shear_range=0.2,zoom_range=0.5,horizontal_flip=True,fill_mode='nearest',validation_split=0.33)

val_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.33 )

IMAGE_RES = 299
BATCH_SIZE = 128

training_set = train_datagen.flow_from_dataframe(dataframe=train,x_col='path',y_col= "labels",
target_size = (IMAGE_RES, IMAGE_RES),batch_size = BATCH_SIZE,validate_filenames=False,subset = "training",seed = 12)

validation_set = val_datagen.flow_from_dataframe(dataframe=train,x_col='path',y_col= "labels",target_size = (IMAGE_RES, IMAGE_RES),
batch_size = BATCH_SIZE,validate_filenames=False,subset = "validation",seed = 12)

In [ ]:
class_names = training_set.class_indices

print(class_names)

In [ ]:
num_classes = len(class_names)
print(num_classes)

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=1e-3)

EPOCHS = 2

metrics = tfa.metrics.F1Score(num_classes = num_classes,average = "macro",name = "f1_score")
    

# Create a callback that stops fitting when val loss do not decrease
callback = tf.keras.callbacks.EarlyStopping(monitor="val_f1_score", patience=5, mode='max')

reducelr = tf.keras.callbacks.ReduceLROnPlateau( monitor= "val_f1_score",mode='max',factor=0.1,patience=4,verbose=1)

In [ ]:
IMAGE_SIZE=[299,299]

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3


inception = InceptionV3(input_shape=IMAGE_SIZE + [3], include_top=False)

In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(inception.output)
prediction = layers.Dense  (num_classes, activation='sigmoid')(x)
model = Model( inputs=inception.input, outputs=prediction) 

In [ ]:
model.summary()


In [ ]:
model.compile(optimizer=optimizer,loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
metrics=[metrics])

In [ ]:
history = model.fit(training_set,epochs=2,validation_data=validation_set,verbose=1,callbacks=[callback,reducelr])

In [ ]:
acc = history.history['f1_score']
val_acc = history.history['val_f1_score']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(history.history['loss']))

plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training f1_score')
plt.plot(epochs_range, val_acc, label='Validation f1_score')
plt.legend(loc='lower right')
plt.title('Training and Validation f1_score')


plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
class_names = {v: k for k, v in class_names.items()}
class_names

In [ ]:
data_sample = train.sample(500)
data_sample.head()

In [ ]:
full_datagen = ImageDataGenerator(rescale = 1./255)

full_set = full_datagen.flow_from_dataframe(dataframe=data,
x_col='path',y_col= None,target_size = (IMAGE_RES, IMAGE_RES),batch_size = BATCH_SIZE,
shuffle=False,validate_filenames=False,class_mode=None)

In [ ]:
def get_labels(prediction,threshold):
  pred = []
  idx = np.where(prediction>threshold)[0]
  for i in idx:
    pred.append(class_names[i])
  pred = ' '. join(pred)
  if len(pred) == 0:
    pred = []
    idx = np.argmax(prediction)
    pred.append(class_names[idx])
    pred = ' '. join(pred)
    return pred
  else :
    return pred

In [ ]:
predicts = model.predict(full_set) 

In [ ]:
thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

In [ ]:
roc_auc_macro = []
for t in thresholds:
    labels = []
    for i in range(len(predicts)):
        pred = predicts[i]
        labels.append(get_labels(prediction = pred, threshold = t))
    data["pred"]= labels

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(data["labels"])
y_test = mlb.transform(data["labels"])
data["pred"] = data["pred"].str.split(" ")
y_score = mlb.transform(data["pred"]) 

In [ ]:
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))

In [ ]:
mean_tpr = np.zeros_like(all_fpr)
for i in range(num_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

In [ ]:
mean_tpr /= num_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
roc_auc_macro.append(round(roc_auc["macro"],2))
print("Threshold:",t,"ROC: ",round(roc_auc["macro"],2))

In [ ]:
df = pd.DataFrame()
df["thresholds"]= thresholds
df["roc_auc_macro"]= roc_auc_macro
df.sort_values("roc_auc_macro", inplace=True, ascending=False)
df.reset_index(inplace=True, drop = True)
df.head()

In [ ]:
df["thresholds"][0]

In [ ]:
threshold = df["thresholds"][0]

In [ ]:
test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
test = pd.DataFrame(columns = ["image", "labels","path"])
test['image'] = os.listdir(test_dir)

In [ ]:
test.sort_values("image", inplace= True)
test.reset_index(inplace=True, drop=True)

In [ ]:
TEST_PATH = "../input/plant-pathology-2021-fgvc8/test_images"

test["path"] = TEST_PATH + "/" +  test["image"]

test.head()

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_dataframe(dataframe=test,
                                    x_col='path',
                                    y_col= None,                                    
                                    target_size = (IMAGE_RES, IMAGE_RES),
                                    batch_size = BATCH_SIZE,
                                    shuffle=False,
                                    class_mode=None)

In [ ]:
predicts = model.predict(test_set)   

In [ ]:
labels = []
for i in range(len(predicts)):
    pred = predicts[i]
    labels.append(get_labels(prediction = pred, threshold = threshold))
    
test["labels"] = labels
test.head()
submission_1 = test.drop("path", axis = 1)
submission_1.head()

In [ ]:
submission_1.to_csv('submission.csv', index=False)